# Input and Output

In [ ]:
import numpy as np

There are many ways to write files in python. The default way, while general, is not the most suitable for most scientific applications. In science, we tend to have very structured data. We will take advantage of that to simplify our IO (input/output)

## The standard way

Here, first we will open a _file handler_, then write some stuff inside and finally close it. We will not use it again for ther rest of this course.

In [ ]:
#open a file called "test.dat", where we will write some data
f = open("03_standard_way.dat", "w") #here "w" means we are going to *write* the file. You can replace it by "r" to read, and "r+" for appending

In [ ]:
a = np.linspace(0, 1) #Generate some data 
for x in a:
    f.write(str(x) + "\n") # We need to convert the number into a string. We will also add an "end-of-line" character

In [ ]:
f.close() # The file needs to be closed to guarantee all data has been written

Now, lets read this back into our program. We will first read the whole file into an array, and then convert that data into floats.

In [ ]:
f_in = open("03_standard_way.dat", "r")
a = f_in.readlines()
print(a)

Currently, the data is contained in strings. Normally we don't want this, but rather as numbers. We got some converting to do!

In [ ]:
a_float = []
for x in a:
    a_float.append(float(x)) #Convert x to float and then add to the new list
    
a_float = np.array(a_float) #Convert a_float into a numpy array
print(a_float)

### Task: Write a cell that writes a matrix

1. Generate a matrix using `m = np.random.rand(5,6)`
2. Open a file called `03_standard_matrix.dat`
3. Write a loop to write a single row of the matrix. Remember to add an empty space (" ") between entries!
4. Write an external loop to finish each line with an end-of-line ("\n")
5. Close the file



**Bonus task**: Read the matrix back

#### Bonus content:

As this open-use-close structure is so common, Python has the `with` command, which simplifies a bit this loop

Now, the file usage is clearly local, and it is closed automatically! Slightly better, but still a lot of work.


----- End of bonux content -----

That was quite a lot of work to read the file! We can grandly simplify this by using the Numpy functions `savetxt` and `loadtxt`!

## The Numpy way: `savetxt` and `loadtxt`

Numpy has two sibling functions, called `savetxt` and `loadtxt`. The names are rather self explanatory: they save and load data to/from plain text files. 

Plain text has a serious advantage in relation to every other file format: They are extremely simple to read. You can expect your file to be readable in any computer ever produced, and any future produced computer. This is a great advantage, but it also has some disadvantages. First and foremost, it is relatively unstructured, carrying little metadata. Additionally, it can be a surprisingly inneficient way to store data. We will talk about more data formats later.

In [ ]:
a = np.linspace(0, 1) #Generate some data 
np.savetxt("03_python_way.dat", a) #Save our data into a file
print(a)

In [ ]:
b = np.loadtxt("03_python_way.dat")
print(b)

Muuuch easier than before! It only takes a single command to do what took us 5 lines to do beforehand. And that scales well for other formats as well. If you want to do the same thing for a matrix, little would change!

In [ ]:
m = np.random.rand(5, 6)
np.savetxt("03_python_way_matrix.dat", m) #Save our data into a file
print(m)

In [ ]:
m2 = np.loadtxt("03_python_way_matrix.dat")

In [ ]:
print(m2)

### Multiple datasets per file

One of the main disadvantages of the plain-text format is the lack of data organization/metadata. When adding multiple arrays, one can organize them into columns, add a header. This is a good, even if not ideal solution. But what about the case when one has several matrices, perhaps of different sizes? 

In this case, we commonly use the npz format. This file has some advantages over the plain text:

1. It can save the name as well as the data itself
2. It can be compressed, taking less space
3. It is binary, meaning no loss of precision

They are mostly associated with numpy, but there are several libraries to read them in other programming languages as well. 

In [ ]:
matrix1 = np.random.rand(100, 100)
np.savez("03_numpy_way.npz", matrix1) #Write a npz file
print(matrix1)

In [ ]:
data = np.load("03_numpy_way.npz")

The data returned in called a dictionary. We can check which datasets we have by 

In [ ]:
print(list(data.keys()))

But that is not the original name! Unfortunately, Python cannot automatically get the name of the variables. Thus, we need to specify when saving.

In [ ]:
np.savez("03_numpy_way.npz", saved_matrix=matrix1) #Write a npz file
data = np.load("03_numpy_way.npz")
print(list(data.keys()))

Now we are talking! All left for now is to print the saved matrix.

In [ ]:
matrix = data['saved_matrix']
print(matrix)

Lets now do this for several matrices. 

In [ ]:
matrix1 = np.random.rand(100, 100)
matrix2 = np.random.rand(30, 52)
matrix3 = np.random.rand(99, 128)
matrix4 = np.random.rand(300, 1000)
np.savez("03_numpy_way_multi.npz", matrix1=matrix1, matrix2=matrix2, matrix3=matrix3, matrix4=matrix4)

In [ ]:
data = np.load("03_numpy_way_multi.npz")
print(list(data.keys()))

The gang is all here. If you are using only python, I strongly recommmend you consitently use plain files and/or npz files. However, the reality is that often another file format for data is commonly used: Excel files. They are not completely trivial to import in python, but we can do it. For this, we will use the Pandas library.

## Reading excel files

In order to read/write data from Excel, we will use the `Pandas` library. This works by leveraging the `DataFrame` structure of Pandas. You can think of this data structure as a mix between the arrays from Numpy and Excel data. The largest change is that the `DataFrames` can have named rows/columns, which make them really useful for data science. As always, we want to keep the metadata close to the data.

This is a good moment to open the file `03_scaling.xlsx` and explore its contents.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_excel("./03_scaling.xlsx")

In [ ]:
data

Now, we can select any given column using its name.

In [ ]:
cores = data["Cores"]

Sometimes, we actually need a subsection of the full data. For this, we can use the `loc` method. This works very similarly to the array indexing in Numpy

In [ ]:
data.loc[0, :]

### Task: Select the speedup for `cores <= 4`

### Files with more than one sheet

Frequently, we aggregate several sheets into a single Excel file. By default, Pandas will read the first one of these, which is not necessarily what we want. There are several ways to handle this. 

First, we can tell Pandas to read the $i^{th}$ sheet

In [ ]:
data = pd.read_excel("./03_scaling_multiple.xlsx", sheet_name=1)

In [ ]:
data

We can also explictly choose the sheet by its name

In [ ]:
data = pd.read_excel("./03_scaling_multiple.xlsx", sheet_name="System2")

In [ ]:
data

Even better, we can read the several sheets at once!

In [ ]:
datas = pd.read_excel("./03_scaling_multiple.xlsx", sheet_name=[0,1])

In [ ]:
data0 = datas[0]
data0

In [ ]:
data1 = datas[1]
data1

### Task: Try to do load both sheets at once, but using the name of the sheets: "System1" and "System2"

**Bonus**: Try passing `None` to `sheet_name`

## Manipulating the DataFrame

As mentioned before, `DataFrame`s work in the same way a normal Numpy array

In [ ]:
data = pd.read_excel("./03_scaling.xlsx")
data["Cores"]+= 1

In [ ]:
data

## Saving an Excel file

Now that we manipulated the data, we can save it into a new file

In [ ]:
data.to_excel("03_scaling_extracores.xlsx", sheet_name="System1")

However, with this method we can only save a single sheet per file. We can overcome this issue with the help of `ExcelWriter`, an utility class from Pandas

In [ ]:
# Read some data, just be sure we have something to work with
# Normally, one would have manipulated the data before simply
# rewriting
datas = pd.read_excel("./03_scaling_multiple.xlsx", sheet_name=["System1", "System2"]) 

In [ ]:
with pd.ExcelWriter('03_scaling_extracores_multiple.xlsx') as writer:
    datas["System1"].to_excel(writer, sheet_name="CoolNewSystem1")
    datas["System2"].to_excel(writer, sheet_name="NotSoCoolSystem2")